# Сравнение аппроксимации задачи явной и неявной схемой

In [4]:
### Пояснялки в комментариях
from matplotlib import pyplot as plt
### Пишешь не в jupyter - комментируй эту строку
%config InlineBackend.figure_formats = ['svg']
### Эту тоже
from ipywidgets import interact
### Это самая важная в мире библиотека без нее никуда
import numpy as np
import scipy as sp

In [8]:
# Реализация метода прогонки для двудиагональной матрицы
def mysolve(A, b):
    m = len(b)
    U = np.zeros(m)
    U[0] = b[0]
    for k in range(1, m):
        b_k = -A[k, k-1]
        a_k = A[k, k]
        xi_k = b[k]
        U[k] = (b_k*U[k-1]+xi_k)/a_k
    return U
# Решение задачи Неявной схемой
def solve_implicit(N, K, Tmax, Xmax):
    # На каждом шаге будем составлять матрицу описанного вида
    # Затем получать решения для описанных методов в предыдущем блоке
    # Вычисляем наши шаги, по факту они практически не нужны
    tau = Tmax/K; h = Xmax/N;
    
    # Тут запишем чего мы ждем - матрицу K*N
    U = np.zeros((K, N))
    
    # Заданим массив из Х от 0 до 1 из N точек, шаг эта штука считает сама
    X = np.linspace(0, Xmax, N)
    # Зададим массив из T
    T = np.linspace(0, Tmax, K)
    
    # Тут учтем кравевые условия
    U[0, :] = ux0(X)
    U[:, 0] = u0t(T)
    
    # initiate SLE matrix
    A = np.zeros((N, N))
    
    # initiate SLE b
    b_my = np.zeros(N)

    # coner matrix elem
    A[0, 0] = 1
    for k in range(0, K-1):
        # Зададим элменты на диагонали матрицы
        # Для этого срежем из матрицы первую строку и первый столбец
        A[1:, 1:][np.diag_indices_from(A[1:, 1:])] = 1+c(X[1:])*tau/h
        
        # Зададим элменты на побочной диагонали
        # Для этого срежем из матрицы первую строку и последний столбец
        # Побочная диагональ переедет на главнуюю
        A[1:, :-1][np.diag_indices_from(A[1:, :-1])] = -(c(X[1:])*tau/h)
        
        # Сотавим вектор правой части
        b_my[0] = U[k+1, 0]
        b_my[1:] = U[k, 1:] + tau*f(X[1:], tau*k)
        
        # Решаем СЛУ для нахождения k+1 слоя
        U[k+1] = mysolve(A, b_my)
    return X, T, U

# Решение явной схемой
def solve_explicit(N, K, Tmax, Xmax):
    # Вычисляем наши шаги, по факте они практически не нужны
    tau = Tmax/K; h = Xmax/N;
    # Тут запишем чего мы ждем - матрицу K*N
    U = np.zeros((K, N))
    # Заданим массив из Х от 0 до 1 из N точек, шаг эта штука считает сама
    X = np.linspace(0, Xmax, N)
    # Зададим массив из T
    T = np.linspace(0, Tmax, K)
    # Тут учтем кравевые условия
    U[0, :] = ux0(X)
    U[:, 0] = u0t(T)
    # Погнали решать разностную задача для неявлной схемы
    for k in range(0, K-1):
        # Производная по х вычислена сразу для всего временного слоя U[k, 1:] - U[k, :-1] вот этой штукой
        # Очконул, что написано в квадратных скобках? - ботай срезы списков
        U[k+1, 1:] = (f(X[1:], tau*(k)) - c(X[1:])*(U[k, 1:] - U[k, :-1])/h)*tau + U[k, 1:]
        # Вернем все, что нам надо
    return (X, T, U)

опредедим константы в задаче

In [45]:
# Тут задаем начальное условие
def ux0(x):
    return np.exp(-(x-0.2)**2/0.25)

# Тут задаем краевое условие при x = 0
def u0t(t):
    return 0.1*np.sin(2*t)
    
# Неоднородная часть
def f(x, t):
    omega = 5
    return 0.1*np.sin(omega*t)*np.cos(x)

# Зависимость скорости от координаты
def c(x):
    return 1.1 - 0.8*np.exp(-x**2/25)#1/(1+np.exp(x))

рисовалка

In [46]:
# Решаем уравнение обоими методами
K = 10000
N = 1000
Tmax = 50
Xmax = 80
X, T, sol_imp = solve_implicit(N, K, Tmax, Xmax)
X, T, sol_exp = solve_explicit(N, K, Tmax, Xmax)
H = -np.sqrt(c(X)) # Интересный факт: скорость волны пропорциональна корню из глубины

In [44]:
# Функция строит оба решения в момент времени t профиль дна бассейна, разность решений
def var(t):
    n = int(t*K/Tmax)%K
    fig = plt.figure(figsize=(12, 4))
    fig.suptitle(rf"$\tau$={Tmax/K}, $h$ = {Xmax/N}")
    # В заголовке написан коэффицент на который написана устойчивость
    plt.subplot(121)
    plt.plot(X, sol_exp[n], label=f"explict")
    plt.plot(X, sol_imp[n], label=f"implict")
    plt.plot(X, H, color="black", label=r"$H(x)$")
    plt.legend()
    plt.subplot(122)
    plt.plot(X, sol_exp[n]-sol_imp[n])
    plt.title(r"$\Delta u$")
    plt.show()
    return None 
interact(var, t=(0, Tmax, 0.1))

interactive(children=(FloatSlider(value=25.0, description='t', max=50.0), Output()), _dom_classes=('widget-int…

<function __main__.var(t)>